In [1]:
from vivarium.interface import setup_simulation

from mslt_port import data
from mslt_port.population import BasePopulation, Mortality, Disability
from mslt_port.disease import Disease

/home/james/miniconda3/envs/vivarium/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/james/miniconda3/envs/vivarium/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
config = data.get_default_config()
sim = setup_simulation([BasePopulation(), Mortality(), Disability(), Disease('chd')], config)

/home/james/miniconda3/envs/vivarium/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [3]:
sim.run()

110

In [4]:
sim.population.population

,tracked,age,sex,population,chd_C_intervention,chd_C,chd_S_intervention,chd_S_intervention_previous,chd_S_previous,chd_C_previous,chd_S,chd_C_intervention_previous
0,False,110,female,7.087417,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
1,False,110,male,3.074316,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
2,False,110,female,7.112880,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
3,False,110,male,3.087840,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
4,False,110,female,7.115229,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
5,False,110,male,3.088891,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
6,False,110,female,7.116581,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
7,False,110,male,3.089508,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
8,False,110,female,7.117791,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
9,False,110,male,3.090034,0.000000,0.000000,1000.000000,1000.000000,1000.000000,0.000000,1000.000000,0.000000
